## 3.1 MLP Data analysis
[More data](http://archive.ics.uci.edu/ml/index.php)

In [1]:
import numpy as np
import tensorflow as tf
np.set_printoptions(suppress=True)

np.set_printoptions(suppress=True):  
precision : int, optional， Number of digits of precision for floating point output (default 8)）  

threshold : int, optional，Total number of array elements which trigger summarization rather than full repr (default 1000).）  

edgeitems : int, optional，Number of array items in summary at beginning and end of each dimension (default 3)  .

linewidth : int, optional，The number of characters per line for the purpose of inserting line breaks (default 75).

suppress : bool, optional，Whether or not suppress printing of small floating point values using scientific notation (default False).  

nanstr : str, optional，String representation of floating point not-a-number (default nan).  

infstr : str, optional，String representation of floating point infinity (default inf).  

np.set_printoptions(threshold=np.nan)  


## 3.2 Data preprocessing

In [2]:
with open('data/lenses/lenses.data','r') as file:
    datas = file.readlines()
    X = []
    Y = []
    for data in datas:
        data_list = data.split()
        X.append(data_list[1:5])
        Y.append(data_list[5])
train_X, train_Y, test_X, test_Y = np.array(X[0:20]), np.array(Y[0:20]), np.array(X[20:]), np.array(Y[20:])
print(train_X)
#tain_X = np.array(X)
#train_Y = np.array(Y)
#test_X = np.array([X[3], X[7], X[15], X[23]])
#test_Y = np.array([Y[3], Y[7], Y[15], Y[23]])
#use the onehot to preprocess data
def onehot(label):
    onehot_label = []
    for index in range(label.shape[0]):
        onehot = np.zeros([3])
        onehot[int(label[index])-1] = 1
        onehot_label.append(onehot)
    return np.array(onehot_label,dtype='float32')
train_label = onehot(train_Y)
test_label = onehot(test_Y)

[['1' '1' '1' '1']
 ['1' '1' '1' '2']
 ['1' '1' '2' '1']
 ['1' '1' '2' '2']
 ['1' '2' '1' '1']
 ['1' '2' '1' '2']
 ['1' '2' '2' '1']
 ['1' '2' '2' '2']
 ['2' '1' '1' '1']
 ['2' '1' '1' '2']
 ['2' '1' '2' '1']
 ['2' '1' '2' '2']
 ['2' '2' '1' '1']
 ['2' '2' '1' '2']
 ['2' '2' '2' '1']
 ['2' '2' '2' '2']
 ['3' '1' '1' '1']
 ['3' '1' '1' '2']
 ['3' '1' '2' '1']
 ['3' '1' '2' '2']]


## 3.3 Setting parameters

In [3]:
input_num = 4
h1_num = 64
h2_num = 32
label_num = 3
epoch = 300

W1 = tf.Variable(tf.truncated_normal([input_num, h1_num], stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([h1_num, h2_num]))
W3 = tf.Variable(tf.truncated_normal([h2_num,label_num]))
B1 = tf.Variable(tf.zeros([h1_num]))
print(W1,W2,W3,B1)
B2 = tf.Variable(tf.zeros([h2_num]))
B3 = tf.Variable(tf.zeros([label_num]))
x = tf.placeholder(tf.float32, [None, input_num])
label = tf.placeholder(tf.float32, [None,label_num])

<tf.Variable 'Variable:0' shape=(4, 64) dtype=float32_ref> <tf.Variable 'Variable_1:0' shape=(64, 32) dtype=float32_ref> <tf.Variable 'Variable_2:0' shape=(32, 3) dtype=float32_ref> <tf.Variable 'Variable_3:0' shape=(64,) dtype=float32_ref>


## 3.4 Build model

In [4]:
# Feedforword
hidden1 = tf.nn.relu(tf.matmul(x, W1) + B1)
hidden2 = tf.nn.relu(tf.matmul(hidden1, W2) + B2)
y = tf.matmul(hidden2, W3) + B3
# Backforword
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=y, labels=label)
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

## 3.5 Train and Test

In [5]:
with tf.Session() as sess:
    # Train
    tf.global_variables_initializer().run()
    for _ in range(epoch):
        sess.run(train_step, feed_dict={x: train_X, label: train_label})
    # Test
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(label,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    predict_acc, loss_= sess.run([accuracy, cross_entropy], feed_dict={x: test_X, label: test_label})
    print("The accuracy of the test data：{:.2f}%".format(predict_acc * 100))

The accuracy of the test data：75.00%


In [6]:
loss_ 

array([0.03218256, 1.2713385 , 0.00713207, 0.03755781], dtype=float32)